In [1]:




import matplotlib.pyplot as plt
import random

import os

import datasets
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import multiprocessing as mp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import io, transforms
from torch.utils.data import Dataset, DataLoader, random_split

from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
from transformers import VisionEncoderDecoderModel , ViTFeatureExtractor
from transformers import AutoTokenizer ,  GPT2Config , default_data_collator


if torch.cuda.is_available():    

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

%autosave 1

There are 1 GPU(s) available.
We will use the GPU: Quadro P5000


Autosaving every 1 seconds


In [2]:
# !pip install transformers matplotlib joblib opencv-python scikit-learn datasets rouge-score
# !pip install joblib
# !pip install gdown

In [3]:
# import gdown

# # url = "https://drive.google.com/u/0/uc?id=1vrXaLX7k9jyLXeSCinxA6msUyKDrQTvz&export=download"
# url = "https://drive.google.com/u/0/uc?id=1U-tbS_gRI3dKTBQ0OW6-xPLnJRGoOXFm&export=download"
# output = "data_3.zip"
# gdown.download(url, output)

In [4]:
# !unzip ./data_3.zip

In [5]:
import joblib

df_train = joblib.load('df_train.pkl')
df_test = joblib.load('df_test.pkl')
df_train.head()

,index,path_properties,path_images,comp_type,comp_label,name,token_single,token_dataset,text
0,0,/home/vamsidhar/Documents/purecode/data_3/trai...,/home/vamsidhar/Documents/purecode/data_3/trai...,Switch,0,MfaISCBZ87B55HsM6Mc6i,"[input_ids, attention_mask]",NaN,"{""componentType"": ""Switch"", ""componentProps/de..."
1,1,/home/vamsidhar/Documents/purecode/data_3/trai...,/home/vamsidhar/Documents/purecode/data_3/trai...,Switch,0,TU60pICpP--GkI5DFnfSy,"[input_ids, attention_mask]",NaN,"{""componentType"": ""Switch"", ""componentProps/di..."
2,2,/home/vamsidhar/Documents/purecode/data_3/trai...,/home/vamsidhar/Documents/purecode/data_3/trai...,Switch,0,uqMDZbdXuZrM6Xfiie1ph,"[input_ids, attention_mask]",NaN,"{""componentType"": ""Switch"", ""componentProps/de..."
3,3,/home/vamsidhar/Documents/purecode/data_3/trai...,/home/vamsidhar/Documents/purecode/data_3/trai...,Switch,0,oCn9YXPyrIyQdaujio-Pe,"[input_ids, attention_mask]",NaN,"{""componentType"": ""Switch"", ""componentProps/de..."
4,4,/home/vamsidhar/Documents/purecode/data_3/trai...,/home/vamsidhar/Documents/purecode/data_3/trai...,Switch,0,S3OgTsHHCPlhZn9L49OLt,"[input_ids, attention_mask]",NaN,"{""componentType"": ""Switch"", ""componentProps/ch..."


In [6]:
df_train['path_properties'] = df_train.path_properties.apply(lambda x: x.replace('/home/vamsidhar/Documents/purecode/', './'))
df_train['path_images'] = df_train.path_images.apply(lambda x: x.replace('/home/vamsidhar/Documents/purecode/', './'))
df_train.path_images[0]

'./data_3/train/images/Switch/MfaISCBZ87B55HsM6Mc6i.png'

In [7]:
df_test['path_properties'] = df_test.path_properties.apply(lambda x: x.replace('/home/vamsidhar/Documents/purecode/', './'))
df_test['path_images'] = df_test.path_images.apply(lambda x: x.replace('/home/vamsidhar/Documents/purecode/', './'))
df_test.path_images[0]

'./data_3/test/images/Card/hBCqHmQDKXBB7f0tka2Mg.png'

In [8]:
notebook_name = None

In [9]:
%%javascript
IPython.notebook.kernel.execute('notebook_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [10]:
# %%javascript
# var kernel = IPython.notebook.kernel;
# var thename = window.document.getElementById("notebook_name").innerHTML;
# var command = "notebook_name = " + "'"+thename+"'";
# kernel.execute(command);


In [11]:
if notebook_name is None:
    notebook_name = 'multi_modal.ipynb'
print(notebook_name)

multi_modal.ipynb


In [12]:
# 

In [13]:
from time import gmtime, strftime
session_key = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(session_key)

2023-04-02-17-55-57


In [14]:
# import wandb

# os.environ['WANDB_NOTEBOOK_NAME'] = notebook_name

# wandb.init(project="component-properties-generation",
#         config={
#             "learning_rate": None,
#             "epochs":  None,
#             "batch_size":  None,
#             "val_split":  None,
#             "dropout":  None,
#             "image_width":  None,
#             "image_height":  None,
#             "random_seed":  None,
#             "session_key":  None,
#             "model_name": "ResNet50V2",
#             "train_accuracy": None,
#             "val_accuracy": None,
#             "test_accuracy": None,
#             "class_weight_strategy": None,
#             "fine_tuning": None,
#             "pad_input": None,
#             "pad_strategy":None
#         })

# w_config = wandb.config



In [15]:
# w_config.learning_rate = 0.001
# w_config.epochs = 2
# w_config.batch_size = 32
# w_config.val_split = 0.2
# w_config.dropout = 0.1
# w_config.image_width = 128
# w_config.image_height = 128
# w_config.random_seed = 42
# w_config.session_key = session_key
# w_config.model_name = "resnet50v2_gpt2"
# w_config.train_accuracy = 0
# w_config.val_accuracy = 0
# w_config.test_accuracy = 0
# w_config.class_weight_strategy = "Augmented"    #keys: None, "Original", "Augmented"
# w_config.fine_tuning = False                     #Fine-tunes the model if True
# w_config.pad_input = True
# w_config.pad_strategy = 'pad_only'      #keys: 'pad_only', 'scale_and_pad'

Organizing data for model training and inferencing



Model training will happen on images from train_folder <br>
train_folder = Training set (Will be split into train and validation to be used for model training)

Model inferencing will happen on images from test_folder <br>
test_folder = Test set (Will be used to evaluate the trained model for <b>inference</b>)



In [16]:

shuffle_dataset = True
random_seed = 42


save_folder = '/home/vamsidhar/Documents/purecode/solution_1/test_' + str(session_key)
print(save_folder)
os.makedirs(save_folder, exist_ok=True)

model_save_path = os.path.join(save_folder, "model_weights.h5")
architecture_save_path = os.path.join(save_folder, "model_architecture.txt")


np.random.seed(random_seed)
random.seed(random_seed)



train_df_path = '/home/vamsidhar/Documents/purecode/solution_1/df_train.pkl'
test_df_path = '/home/vamsidhar/Documents/purecode/solution_1/df_test.pkl'

train_folder = '/home/vamsidhar/Documents/purecode/data_3/train/images'
test_folder = '/home/vamsidhar/Documents/purecode/data_3/test/images'

tokenizer_path = '/home/vamsidhar/Documents/purecode/solution_1/tokenizer.pkl'



/home/vamsidhar/Documents/purecode/solution_1/test_2023-04-02-17-55-57


In [17]:
# import joblib

# df_train = joblib.load(train_df_path)
# df_test = joblib.load(test_df_path)



In [18]:


def get_weights(df):
    cls = df.comp_label.unique()
    
    cls_count = df.comp_label.value_counts()
    maximum = max(cls_count)
    cls_count = maximum/cls_count
    
    weights = cls_count.to_dict()


    return weights





In [19]:
# print(max(df_train.comp_label.value_counts()))
# a = 4636/df_train.comp_label.value_counts()
# a.to_dict()

In [20]:

classes_train = df_train.comp_type.unique()
classes_test  = df_test.comp_type.unique()
print(classes_train)

num_classes = len(classes_train)


class_weights = get_weights(df=df_train)
    
print(class_weights)


['Switch' 'Checkbox' 'Card' 'Icon' 'Button']
{3: 1.0, 4: 1.758058399696625, 2: 2.136405529953917, 1: 3.6049766718506997, 0: 14.578616352201259}


In [21]:
   


def generate_filenames_label(df, shuffle=True, split=False, split_ratio=0.2):
    if not split:
        file_name_img = df.path_images.values
        properties = list(df.text.values)
#         label = label_encoding(df)
        label = df.comp_label.values
    
        return label, properties, file_name_img
    
    else:
        df_temp = df.copy()
        
        df_val = df_temp.sample(frac = 0.2, random_state=42)
        df_train = df_temp.drop(df_val.index)

        df_val.reset_index(inplace=True)
        df_train.reset_index(inplace=True)

        file_name_img_train = df_train.path_images.values
        properties_train = list(df_train.text.values)
#         label_train = label_encoding(df_train)
        label_train = df_train.comp_label.values
        
        
        file_name_img_val = df_val.path_images.values
        properties_val = list(df_val.text.values)
#         label_val = label_encoding(df_val)
        label_val = df_val.comp_label.values
        
        
        return label_train, properties_train, file_name_img_train, label_val, properties_val, file_name_img_val  


    
    

In [22]:
label_train, prop_train, file_name_train, label_val, prop_val, file_name_val = generate_filenames_label(df_train,
                                                                                  shuffle=True,
                                                                                  split=True,
                                                                                  split_ratio=0.2)



In [23]:
label_test, prop_test, file_name_test  = generate_filenames_label(df_test, shuffle=False, split=False)



In [24]:
from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
from transformers import TFVisionEncoderDecoderModel , ViTFeatureExtractor
from transformers import AutoTokenizer ,  GPT2Config , default_data_collator

import multiprocessing as mp

In [25]:
class config : 
    ENCODER = "google/vit-base-patch16-224"
    DECODER = "gpt2"
    TRAIN_BATCH_SIZE = 24
    VAL_BATCH_SIZE = 24
    VAL_EPOCHS = 1
    LR = 5e-5
    SEED = 42
    MAX_LEN = 128
    SUMMARY_LEN = 20
    WEIGHT_DECAY = 0.01
    MEAN = (0.485, 0.456, 0.406)
    STD = (0.229, 0.224, 0.225)
    TRAIN_PCT = 0.95
    NUM_WORKERS = mp.cpu_count()
    EPOCHS = 3
    IMG_SIZE = (224,224)
    LABEL_MASK = -100
    TOP_K = 1000
    TOP_P = 0.95

In [26]:
from transformers import AutoTokenizer ,  GPT2Config , default_data_collator

def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs
AutoTokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens




In [27]:
import datasets

rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [28]:
feature_extractor = ViTFeatureExtractor.from_pretrained(config.ENCODER)
tokenizer = AutoTokenizer.from_pretrained(config.DECODER)
tokenizer.pad_token = tokenizer.unk_token

# tokenizer = joblib.load(tokenizer_path)

In [29]:
# import cv2
# img = cv2.imread(df_train.path_images[0])/255

# feature_extractor([img, img, img], return_tensors='tf').pixel_values


In [30]:
class ImgDataset(Dataset):
    def __init__(self, filenames, labels, props, tokenizer,
                 feature_extractor, shuffle=True):
        
        self.labels = labels
        self.filenames = filenames
        self.props = props
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        
        
        
#         self.df = df
#         self.transform = transform
#         self.root_dir = root_dir
        self.max_length = 128
        
    def __len__(self,):
        return len(self.filenames)
    
    def __getitem__(self,idx):
        caption = self.props[idx]
        img_path = self.filenames[idx]
#         img = self.__load_image__(img_path)
        img = Image.open(img_path).convert("RGB")
        
#         if self.transform is not None:
#             img= self.transform(img)
        pixel_values = self.feature_extractor(img, return_tensors="pt").pixel_values
        captions = self.tokenizer(caption,
                                 padding='max_length',
                                  truncation=True,
                                 max_length=self.max_length).input_ids
        
        captions = [caption if caption != self.tokenizer.pad_token_id else -100 for caption in captions]
        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(captions)}
        return encoding
    
    
#     def __load_image__(self, image_path):
#         img = cv2.imread(image_path)
#         img = self.__resize_image__(img, pad=True)
            
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img = img.astype(np.float32) / 255.0
        
        
#         return img

#     def __resize_image__(self, img, pad=True):
#         old_size = img.shape[:2]
#         ratio = float(self.dim[0])/max(old_size)
#         new_size = tuple([int(x*ratio) for x in old_size])
#         img_new = cv2.resize(img, (new_size[1], new_size[0])) 

#         #resize by padding
#         delta_w = self.dim[1] - new_size[1]
#         delta_h = self.dim[0] - new_size[0]
#         top, bottom = delta_h//2, delta_h-(delta_h//2)
#         left, right = delta_w//2, delta_w-(delta_w//2)

#         color = [0, 0, 0]
#         img_new = cv2.copyMakeBorder(img_new, top, bottom, left, right, cv2.BORDER_CONSTANT,
#             value=color)

        
#         return img_new



In [31]:
train_dataset = ImgDataset(filenames=file_name_train, labels=label_train, props=prop_train,
                           tokenizer=tokenizer,feature_extractor = feature_extractor )

val_dataset = ImgDataset(filenames=file_name_val, labels=label_val, props=prop_val,
                           tokenizer=tokenizer,feature_extractor = feature_extractor )

test_dataset = ImgDataset(filenames=file_name_test, labels=label_val, props=prop_test,
                           tokenizer=tokenizer,feature_extractor = feature_extractor )





In [32]:
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(config.ENCODER, 
                                                                  config.DECODER)

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly ini

In [33]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size
# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.max_length = 128
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [34]:
training_args = Seq2SeqTrainingArguments(
    output_dir='VIT_large_gpt2',
    per_device_train_batch_size=config.TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=config.VAL_BATCH_SIZE,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=1024,  
    save_steps=2048, 
    warmup_steps=1024,  
    learning_rate = 5e-5,
    #max_steps=1500, # delete for full training
    num_train_epochs = config.EPOCHS, #TRAIN_EPOCHS
    overwrite_output_dir=True,
    save_total_limit=1,
)

In [35]:
# import torch
# torch.cuda.empty_cache()

In [36]:
from transformers import default_data_collator
import cv2

trainer = Seq2SeqTrainer(
    tokenizer=feature_extractor,
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
)


In [37]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8838
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 1107
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: vamsidhar_muthireddy. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
1,No log,0.447180,0.124100,0.332300,0.165400
2,No log,0.305749,0.129000,0.340100,0.171900
3,0.710500,0.223165,0.147300,0.375500,0.195100


***** Running Evaluation *****
  Num examples = 2209
  Batch size = 24
***** Running Evaluation *****
  Num examples = 2209
  Batch size = 24
***** Running Evaluation *****
  Num examples = 2209
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1107, training_loss=0.6768736163021318, metrics={'train_runtime': 8817.2323, 'train_samples_per_second': 3.007, 'train_steps_per_second': 0.126, 'total_flos': 4.784819609159074e+18, 'train_loss': 0.6768736163021318, 'epoch': 3.0})

In [43]:
trainer.save_model('VIT_large_gpt2')

Saving model checkpoint to VIT_large_gpt2
Configuration saved in VIT_large_gpt2/config.json
Model weights saved in VIT_large_gpt2/pytorch_model.bin
Feature extractor saved in VIT_large_gpt2/preprocessor_config.json


In [49]:
import joblib

joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [48]:
import joblib
joblib.dump(feature_extractor, "feature_extractor.pkl")

['feature_extractor.pkl']

In [47]:
import joblib
joblib.dump( model, "model.pkl")

['model.pkl']

In [59]:
test_index = 1

In [60]:
from PIL import Image
test_image_path = df_test.path_images[test_index]
img =  Image.open(test_image_path).convert("RGB")

In [70]:
# generated_caption = tokenizer.decode(model.generate(feature_extractor(img, return_tensors="pt").pixel_values.to("cuda"))[0])
generated_caption = tokenizer.decode(model.generate(feature_extractor(img, return_tensors="pt").pixel_values.to("cuda"))[0], skip_special_tokens=True)

print(generated_caption)

{"componentType": "Button", "componentProps/sx/color": "#ffffff", "completed": false, "committed": true, "componentStatus": "inherit", "sx/_textTransform": "none", "my": "0.3rem", "m": 1, "mySpacing": "1.5px", "mx": 2, "border": "2px solid #b5b568b5", "borderRadius": "10px", "-webkit-fontSize": "13px", -webkit-textTransform: "none"}, "


In [71]:
df_test.text[test_index]

'{"componentType": "Button", "componentProps/variant": "outlined", "componentProps/sx/color": "grey", "componentProps/sx/bgcolor": "white", "componentProps/sx/border": "1px solid grey", "componentProps/sx/textTransform": "none", "componentProps/sx/p": "0.25rem 5rem 0.25rem 0.35rem", "componentProps/sx/fontSize": "18px", "componentProps/sx/fontWeight": 500, "componentProps/sx/width": "189.078125px", "componentProps/sx/height": "41.5px", "componentProps/size": "large", "componentProps/disableElevation": true}'

In [73]:
generated_caption = tokenizer_loaded.decode(model_loaded.generate(feature_extractor_loaded(img, return_tensors="pt").pixel_values.to("cuda"))[0], 
                                            skip_special_tokens=True)

print(generated_caption)

{"componentType": "Button", "componentProps/sx/color": "#ffffff", "completed": false, "committed": true, "componentStatus": "inherit", "sx/_textTransform": "none", "my": "0.3rem", "m": 1, "mySpacing": "1.5px", "mx": 2, "border": "2px solid #b5b568b5", "borderRadius": "10px", "-webkit-fontSize": "13px", -webkit-textTransform: "none"}, "
